In [2]:
# download nltk stopwords
import nltk
from nltk.corpus import wordnet
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [53]:

import re
from nltk.corpus import wordnet

# Define your regular expression pattern
RE_WORD = re.compile(r"\b[\w'-]{2,24}\b", re.UNICODE)

def get_synonyms(word):
    synonyms = []
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            # Get the normalized form of the word from the lemma
            normalized_word = lemma.name().lower()
            # Check if the word contains any signs
            if normalized_word.isalpha():
                # Remove characters specified by the regular expression pattern
                normalized_word = ' '.join(RE_WORD.findall(normalized_word))
                if normalized_word:
                    synonyms.append(normalized_word)
    return synonyms




['genetics']

In [3]:
# Install a particular version of `google-cloud-storage` because (oddly enough)
# the  version on Colab and GCP is old. A dependency error below is okay.
!pip install -q google-cloud-storage==1.43.0

In [4]:
# authenticate below for Google Storage access as needed
from google.colab import auth
auth.authenticate_user()

In [5]:
!pip install pyngrok

In [6]:
# TODO: sign up for an ngrok account
# then put your ngrok token below and execute
from pyngrok import ngrok
ngrok.set_auth_token("2cXd1YwtWlYn3kGKfj97UtzuCER_42NZiY3AoMj93W18LwJWp")

# Run the app

In [7]:
# you need to upload your implementation of search_app.py
import search_frontend as se
import sim_calc

In [17]:
# uncomment the code below and execute to reload the module when you make
# changes to search_frontend.py (after you upload again).
import importlib
importlib.reload(sim_calc)
importlib.reload(se)


<module 'search_frontend' from '/content/search_frontend.py'>

In [18]:
public_url = ngrok.connect("5000", bind_tls=False).public_url
split_url = public_url.split("http://")
# Check if the split operation resulted in more than one part
if len(split_url) > 1:
    # Split the domain part by ".ngrok.io" to extract the subdomain
    subdomain = split_url[1].split(".ngrok.io")[0]
    print(subdomain)
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")
# Update any base URLs to use the public ngrok URL
se.app.config["BASE_URL"] = public_url
se.app.run()

8a8b-34-106-45-248.ngrok-free.app
 * ngrok tunnel "http://8a8b-34-106-45-248.ngrok-free.app" -> "http://127.0.0.1:5000"
 * Serving Flask app 'search_frontend'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [02/Mar/2024 16:36:48] "GET /search?query=genetics HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Mar/2024 16:36:54] "GET /search?query=Who+is+considered+the+"Father+of+the+United+States"? HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Mar/2024 16:36:55] "GET /search?query=economic HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Mar/2024 16:36:58] "GET /search?query=When+was+the+United+Nations+founded? HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Mar/2024 16:37:00] "GET /search?query=video+gaming HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Mar/2024 16:37:02] "GET /search?query=3D+printing+technology HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Mar/2024 16:37:03] "GET /search?query=Who+is+the+author+of+"1984"? HTTP/1.1" 200 -
INFO:werkzeug:127.0.0

# Testing your app

Once your app is running you can query it. You can simply do that by navigating to the URL that ngrok gave you above or through code in a different python session. For example, once the frontend app is running, you can navigate to:
http://YOUR_SERVER_DOMAIN/search?query=hello+world where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io, which is printed above in Colab or that is your external IP on GCP.

The code below shows how to issue a query from python. This is also how our testing code will issue queries to your search engine, so make sure to test your search engine this way after you deploy it to GCP and before submission. Command line instructions for deploying your search engine to GCP are available at `run_frontend_in_gcp.sh`. Note that we will not only issue training queries to your search engine, but also test queries, i.e. queries that you've never seen before.

In [21]:
import json

with open('queries_train.json', 'rt') as f:
  queries = json.load(f)

In [22]:
def average_precision(true_list, predicted_list, k=40):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    precisions = []
    for i,doc_id in enumerate(predicted_list):
        if doc_id in true_set:
            prec = (len(precisions)+1) / (i+1)
            precisions.append(prec)
    if len(precisions) == 0:
        return 0.0
    return round(sum(precisions)/len(precisions),3)

In [23]:
def precision_at_k(true_list, predicted_list, k):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    if len(predicted_list) == 0:
        return 0.0
    return round(len([1 for doc_id in predicted_list if doc_id in true_set]) / len(predicted_list), 3)
def recall_at_k(true_list, predicted_list, k):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    if len(true_set) < 1:
        return 1.0
    return round(len([1 for doc_id in predicted_list if doc_id in true_set]) / len(true_set), 3)
def f1_at_k(true_list, predicted_list, k):
    p = precision_at_k(true_list, predicted_list, k)
    r = recall_at_k(true_list, predicted_list, k)
    if p == 0.0 or r == 0.0:
        return 0.0
    return round(2.0 / (1.0/p + 1.0/r), 3)
def results_quality(true_list, predicted_list):
    p5 = precision_at_k(true_list, predicted_list, 5)
    f1_30 = f1_at_k(true_list, predicted_list, 30)
    if p5 == 0.0 or f1_30 == 0.0:
        return 0.0
    return round(2.0 / (1.0/p5 + 1.0/f1_30), 3)

assert precision_at_k(range(10), [1,2,3] , 2) == 1.0
assert recall_at_k(   range(10), [10,5,3], 2) == 0.1
assert precision_at_k(range(10), []      , 2) == 0.0
assert precision_at_k([],        [1,2,3],  5) == 0.0
assert recall_at_k(   [],        [10,5,3], 2) == 1.0
assert recall_at_k(   range(10), [],       2) == 0.0
assert f1_at_k(       [],        [1,2,3],  5) == 0.0
assert f1_at_k(       range(10), [],       2) == 0.0
assert f1_at_k(       range(10), [0,1,2],  2) == 0.333
assert f1_at_k(       range(50), range(5), 30) == 0.182
assert f1_at_k(       range(50), range(10), 30) == 0.333
assert f1_at_k(       range(50), range(30), 30) == 0.75
assert results_quality(range(50), range(5))  == 0.308
assert results_quality(range(50), range(10)) == 0.5
assert results_quality(range(50), range(30)) == 0.857
assert results_quality(range(50), [-1]*5 + list(range(5,30))) == 0.0


In [24]:
import requests
from time import time
# url = 'http://35.232.59.3:8080'
# place the domain you got from ngrok or GCP IP below.
url = 'http://e300-35-196-67-125.ngrok-free.app'
qs_res = []
for q, true_wids in queries.items():
  duration, ap = None, None
  t_start = time()
  try:
    res = requests.get(url + '/search', {'query': q}, timeout=35)
    duration = time() - t_start
    if res.status_code == 200:
      pred_wids, _ = zip(*res.json())
      rq = results_quality(true_wids, pred_wids)
  except:
    pass

  qs_res.append((q, duration, rq))

NameError: name 'rq' is not defined